In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ✅ Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 저장된 경로 지정
model_path = "/content/drive/MyDrive/models/codebert_finetuned_v2"

# ✅ 모델과 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

print("✅ Google Drive에서 모델 불러오기 완료!")

Mounted at /content/drive
✅ Google Drive에서 모델 불러오기 완료!


In [9]:
# 최신 openai (v1 이상) 방식
from openai import OpenAI

# ⚠️ 파일 업로드할 때는 숨기기
client = OpenAI(api_key="api-key")

def generate_report(code_snippet: str, label: str) -> str:
    prompt = f"""
당신은 보안 분석 전문가입니다.
아래의 소스 코드는 '{label}' 취약점이 감지된 코드입니다.

[취약 코드]
{code_snippet}

이 코드에 대해 다음 내용을 자세히 설명해주세요:
1. 취약점 설명 (왜 위험한가?)
2. 공격자가 악용하는 방법
3. 보완 방법 (보완된 코드 예시 포함)
4. 요약된 보안 권고사항

전문적인 보안 분석 보고서 형태로 출력해주세요.
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for code security analysis."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=2048
    )

    return response.choices[0].message.content


In [10]:
# 예제 취약 코드
code = """
user_input = request.args.get('username')
query = "SELECT * FROM users WHERE name = '" + user_input + "'"
"""

# CodeBERT 예측 결과
predicted_label = "SQL_Injection"

# 리포트 생성
report = generate_report(code, predicted_label)
print("📄 보안 리포트:\n")
print(report)

📄 보안 리포트:

**보안 분석 보고서**

**1. 취약점 설명:**
위의 코드는 'SQL Injection' 취약점이 존재합니다. 이는 사용자로부터 입력받은 데이터를 쿼리에 그대로 삽입하는 방식으로 작성되어 있기 때문에 악의적인 SQL 쿼리를 삽입하여 데이터베이스를 조작하거나 민감한 정보를 노출시킬 수 있는 위험이 있습니다.

**2. 공격자가 악용하는 방법:**
공격자는 사용자 입력란에 SQL 쿼리를 삽입하여 데이터베이스를 조작하거나 민감한 정보를 탈취할 수 있습니다. 예를 들어, 'admin' OR '1'='1'을 입력하여 모든 사용자 정보를 조회할 수 있습니다.

**3. 보완 방법:**
SQL Injection 취약점을 방지하기 위해서는 사용자 입력을 안전하게 처리해야 합니다. Prepared Statement를 사용하여 사용자 입력을 파라미터로 전달하는 방법이 가장 효과적입니다.

**보완된 코드 예시:**
```python
user_input = request.args.get('username')
query = "SELECT * FROM users WHERE name = %s"
cursor.execute(query, (user_input,))
```

**4. 요약된 보안 권고사항:**
- 사용자 입력을 쿼리에 그대로 삽입하지 말고 Prepared Statement를 사용하여 안전하게 처리해야 합니다.
- 입력값을 검증하고 이스케이프 처리하여 SQL Injection 공격을 방어해야 합니다.
- 데이터베이스 접근 권한을 최소화하여 공격자가 악용할 수 있는 영역을 최소화해야 합니다.

이상으로 'SQL Injection' 취약점에 대한 분석 보고서를 마치겠습니다. 추가 질문이 있으시면 언제든지 문의해주세요. 감사합니다.
